In [ ]:
sudo visudo
<user> ALL=(ALL) NOPASSWD: ALL

In [ ]:
import os
import psutil

data = []
for i in psutil.pids():
    try:
        p = psutil.Process(i)
        res = os.popen(f"more /proc/{i}/stat").read()
        res = res.split()
        data.append(
            (
                p.pid, p.name(),
                p.ppid(), res[21],
                res[16],
                res[17],
                p.cpu_percent(),
                p.memory_percent(),
                p.io_counters().read_count,
                p.io_counters().write_count,
                res[22],
                p.status(),
            )
        )
    except:
        continue
for i in data:
    print(i)

(1, 'docker-init', 0, '0', '59', '6', 0.0, 6.0176328678293134e-05, 3391, 2499, '1', 'sleeping')
(6, 'node', 1, '0', '170', '30', 0.0, 0.36572163754232656, 9909, 3735, '11', 'sleeping')
(18, 'tail', 0, '0', '64', '8', 0.0, 0.004362783829176252, 57, 11, '1', 'sleeping')
(28, 'oom_monitor.sh', 6, '0', '17', '56', 0.0, 0.025274058044883113, 35511, 3801, '1', 'sleeping')
(30, 'run.sh', 1, '0', '0', '0', 0.0, 0.01251667636508497, 0, 0, '1', 'sleeping')
(33, 'kernel_manager_proxy', 30, '0', '21', '13', 0.0, 0.1029917865328987, 1172, 761, '5', 'sleeping')
(59, 'python3', 6, '0', '698', '117', 0.0, 0.0, 8741, 3, '1', 'zombie')
(60, 'colab-fileshim.', 6, '0', '76', '13', 0.0, 0.3258247316286182, 1299, 1, '1', 'sleeping')
(77, 'jupyter-noteboo', 6, '0', '434', '66', 0.0, 1.1554456869519065, 5014, 572, '7', 'sleeping')
(78, 'dap_multiplexer', 6, '0', '28', '12', 0.0, 0.0770858770368935, 1397, 1319, '5', 'sleeping')
(443, 'python3', 77, '0', '547', '76', 0.0, 0.7033710177562293, 4226, 525, '14', 'r

In [ ]:
import psutil
import numpy as np
import pickle
import random
import time
import os

# Define the state space
def get_state():
    data = []
    for i in psutil.pids():
        try:
            p = psutil.Process(i)
            res = os.popen(f"more /proc/{i}/stat").read()
            res = res.split()
            data.append(
                (
                    p.pid,
                    p.name(),
                    p.username(),
                    res[20],
                    res[21],
                    psutil.cpu_percent(),
                    psutil.virtual_memory().percent,
                    psutil.disk_usage("/").percent,
                )
            )
        except:
            continue
    return data

# Set the reward value
reward = 1

# Define the Q-table
q_table = {}
try:
    with open("q_table.pkl", "rb") as f:
        q_table = pickle.load(f)
except:
    pass

# Define the hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# Define the action space
action_space = [-20, -10, -5, 0, 5, 10, 20]


# Initialize the state

# Define the epsilon-greedy policy
def epsilon_greedy_policy(state, epsilon):
    if random.uniform(0, 1) < epsilon:
        return random.choice(action_space)
    else:
        if state in q_table:
            return max(q_table[state],
                       key=q_table[state].get)
        else:
            return 0


for i in range(1000):
    # while(True):
    state_space = get_state()
    # Train the Q-learning algorithm
    for j in range(len(state_space)):
        # Set the initial priority
        priority = state_space[j][4]

        # Choose an action based on the epsilon-greedy policy
        action = epsilon_greedy_policy(state_space[j][1::], epsilon)
        try:
            # Apply the action
            priority += action
            psutil.Process(state_space[j][0]).nice(priority)

            # Observe the new state
            new_state = state_space[j + 1][1::]

            # Get the current Q-value
            current_q = q_table.get((state_space[j][1::], action), 0)

            # Get the max Q-value for the new state
            max_future_q = max(q_table.get(new_state, {0: 0}).values())

      # Calculate the new Q-value
      new_q = current_q + alpha * (reward + gamma * max_future_q - current_q)

      # Update the Q-table
      q_table[(state_space[j][1::], action)] = new_q

      # Update the state
      # state_space[j][1::] = new_state

      # Wait for some time
      time.sleep(0.5)
      except:
          pass
      # Save the Q-table
      with open("q_table.pkl", "wb") as f:
          pickle.dump(q_table, f)


IndentationError: ignored

In [ ]:
import numpy as np
import psutil
import pickle
import random
import time
import os

# Define the state space
def get_state():
    data = []
    for i in psutil.pids():
        try:
            p = psutil.Process(i)
            res = os.popen(f"more /proc/{i}/stat").read()
            res = res.split()
            data.append(
                (
                    p.pid,
                    p.name(),
                    p.username(),
                    int(res[20]),
                    int(res[21]),
                    psutil.cpu_percent(),
                    psutil.virtual_memory().percent,
                    psutil.disk_usage("/").percent,
                )
            )
        except:
            continue
    return data

# Set the reward and penalty values
reward = 1
penalty = -1

# Define the Q-table
q_table = {}
try:
    with open("q_table.pkl", "rb") as f:
        q_table = pickle.load(f)
except:
    pass

# Define the hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.3

# Define the action space
# action_space = [-20, -10, -5, 0, 5, 10, 20]
action_space = [0 , 1]
state_space = []
j = 0

# Initialize the state

# Define the epsilon-greedy policy
def epsilon_greedy_policy(state, epsilon):
    if random.uniform(0, 1) < epsilon:
        return random.choice(action_space)
    else:
        if state in q_table:
            return max(q_table[state], key=q_table[state].get)
        else:
            return 0


for i in range(1000):
    # while(True):
    state_space = get_state()
    # Train the Q-learning algorithm
    for j in range(len(state_space)):
        # Set the initial priority
        priority = state_space[j][4]

        # Choose an action based on the epsilon-greedy policy
        action = epsilon_greedy_policy(state_space[j][1::], epsilon)

        # Apply the action
        # priority += action
        priority -= action
        print(f"Priority = {priority}")
        try:
            psutil.Process(state_space[j][0]).nice(priority)
        except:
            continue
        try:
            # Observe the new state
            new_state = state_space[j + 1][1::]
        except:
            pass

        # Get the current Q-value
        current_q = q_table.get((state_space[j][1::], action), 0)

        # Get the max Q-value for the new state
        max_future_q = max(q_table.get(new_state, {0: 0}).values())

        # Calculate the new Q-value
        new_q = current_q + alpha * (reward + gamma * max_future_q - current_q)

        # Update the Q-table
        q_table[(state_space[j][1::], action)] = new_q

        # Update the state
        # state_space[j][1::] = new_state

        # Wait for some time
        time.sleep(0.01)

        # Save the Q-table
        with open("q_table.pkl", "wb") as f:
            pickle.dump(q_table, f)


In [ ]:
import psutil
import pickle
import random
import time
import os

# Define the state space
def get_state():
    data = []
    for i in psutil.pids():
        try:
            p = psutil.Process(i)
            res = os.popen(f"more /proc/{i}/stat").read()
            res = res.split()
            data.append(
                (
                    p.pid,
                    p.ppid,
                    p.name(),
                    int(res[20]),
                    int(res[21]),
                    psutil.cpu_percent(),
                    psutil.virtual_memory().percent
                )
            )
            psutil.cpu_percent(),
            psutil.virtual_memory().percent
        except:
            continue
    return data

# Set the reward and penalty values
reward = 1
penalty = -1

# Define the Q-table
q_table = {}
try:
    with open("q_table.pkl", "rb") as f:
        q_table = pickle.load(f)
except:
    pass

# Define the hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# Define the action space
# action_space = [-20, -10, -5, 0, 5, 10, 19]
action_space = [0, 1]
state_space = []
j = 0

# Initialize the state

def check_state(state, q_table):
  for key in q_table.keys():
      if key[0] == state:
          return True
  return False

# Define the epsilon-greedy policy
def epsilon_greedy_policy(state, epsilon):
    if random.uniform(0, 1) < epsilon:
        return random.choice(action_space)
    else:
        if check_state(state,q_table):
            return max(q_table[state], key=q_table[state].get)
        else:
            return 0

while(True):
    state_space = get_state()
    # Train the Q-learning algorithm
    for j in range(len(state_space)):
        if ((state_space[j][0] != 1) or (state_space[j][1] != 1)):
            # Set the initial priority
            priority = state_space[j][4]

            # Choose an action based on the epsilon-greedy policy
            action = epsilon_greedy_policy(state_space[j][2::], epsilon)

            # Apply the action
            priority -= action
            try:
                #psutil.Process(state_space[j][0]).nice(priority)
                os.popen(f"sudo renice -n {priority} -p {state_space[j][0]}")
            except:
                pass
            try:
                # Observe the new state
                new_state = state_space[j + 1][2::]
            except:
                pass

            # Get the current Q-value
            current_q = q_table.get((state_space[j][2::], action), 0)

            # Get the max Q-value for the new state
            max_future_q = max(q_table.get(new_state, {0: 0}).values())

            # Calculate the new Q-value
            new_q = current_q + alpha * (reward + gamma * max_future_q - current_q)

            # Update the Q-table
            q_table[(state_space[j][2::], action)] = new_q

            # Update the state
            # state_space[j][2::] = new_state

    # Wait for some time
    time.sleep(0.5)

    # Save the Q-table
    with open("q_table.pkl", "wb") as f:
        pickle.dump(q_table, f)

In [ ]:
check_state(('python3', 39, 19), q_table)

In [ ]:
max(q_table[('python3', 39, 19)], key=q_table[('python3', 39, 19)].get)

In [ ]:
q_table[state_space[0][1::]]

In [ ]:
for i in q_table:
  print(i)

In [ ]:
q_table[('python3', 39, 19)]

In [ ]:
print(state_space)

In [ ]:
with open('q_table.pickle', 'rb') as f:
        q_table = pickle.load(f)


In [ ]:
print(q_table)

In [ ]:
import numpy as np
import random
from collections import deque
import tensorflow
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.optimizers import Adam

EPISODES = 100
MAX_TIMESTEPS = 100
BATCH_SIZE = 50

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95  # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = tensorflow.keras.models.Sequential()
        model.add(tensorflow.keras.layers.Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(tensorflow.keras.layers.Dense(24, activation='relu'))
        model.add(tensorflow.keras.layers.Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=tensorflow.keras.optimizers.Adam(learning_rate=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma * np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)


if __name__ == "__main__":
    state_size = 3  # number of state variables to track
    action_size = 5  # number of possible priority values
    agent = DQNAgent(state_size, action_size)

    # user behavior data collection and preprocessing
    # create state variables based on user behavior data
    # normalize state variables (if necessary)

    # training loop
    for e in range(EPISODES):
        # reset environment and initialize state variables
        state = (psutil.virtual_memory().percent, psutil.cpu_percent())
        # time steps within each episode
        for time in range(MAX_TIMESTEPS):
            # choose action
            action = agent.act(state)

            # perform action and get reward
            reward = 1

            # observe new state and store in replay buffer
            next_state = (psutil.virtual_memory().percent, psutil.cpu_percent())
            agent.remember(state, action, reward, next_state, True)

            # update state
            state = next_state

            # perform replay and target model updates
            if len(agent.memory) > BATCH_SIZE:
                agent.replay(BATCH_SIZE)

        # save model weights
        if e % 10 == 0:
            agent.save("model_" + str(e) + ".h5")


In [ ]:
import psutil
from time import sleep
cpu = []
mem = []
for i in range(120):
  cpu.append(psutil.cpu_percent())
  mem.append(psutil.virtual_memory().percent)
  sleep(0.5)
print(cpu)
print(mem)

[12.9, 2.0, 3.0, 2.0, 3.0, 3.0, 3.0, 0.0, 4.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 3.0, 46.5, 56.4, 52.5, 29.0, 1.0, 4.0, 5.1, 2.0, 1.0, 3.0, 1.0, 2.0, 2.0, 5.0, 0.0, 4.0, 3.0, 2.0, 1.0, 2.0, 1.0, 2.0, 2.0, 35.0, 51.5, 53.5, 40.0, 5.1, 2.0, 2.0, 1.0, 2.0, 0.0, 2.0, 2.0, 2.0, 1.0, 3.0, 3.0, 2.0, 2.0, 2.0, 5.0, 3.0, 1.0, 1.0, 22.0, 54.0, 52.5, 55.0, 2.0, 3.0, 1.0, 3.0, 0.0, 3.0, 0.0, 3.0, 1.0, 3.0, 2.0, 1.0, 2.9, 2.0, 1.0, 2.0, 4.0, 2.0, 2.0, 6.1, 58.0, 51.5, 56.6, 12.0, 6.0, 0.0, 2.0, 1.0, 3.0, 0.0, 5.0, 56.0, 53.0, 3.0, 5.1, 3.0, 2.0, 2.0, 2.0, 0.0, 3.0, 5.0, 4.0, 46.5, 57.0, 54.5, 31.0, 2.0, 2.0, 0.0, 4.0, 0.0, 7.9, 0.0]
[7.6, 7.6, 7.6, 7.6, 7.6, 7.6, 7.6, 7.6, 7.6, 7.6, 7.6, 7.6, 7.6, 7.6, 7.6, 7.6, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 7.7, 